In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1579215405741_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [2]:
sc.install_pypi_package("h2o-pysparkling-2.4")
sc.install_pypi_package("h2o")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765/h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e/tabulate-0.8.6-cp36-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/dc/45cdef1b4d119eb96316b3117e6d5708a08029992b2fee2c143c7a0a5cc5/colorama-0.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e/future-0.18.2-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe0285

# H2O

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pysparkling import *
from h2o.automl import H2OAutoML

import pyspark
import h2o

ss = pyspark.sql.SparkSession.builder.master('yarn')\
     .config("spark.dynamicAllocation.enabled", True)\
     .getOrCreate()

hc = H2OContext.getOrCreate(ss)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Connecting to H2O server at http://ip-172-31-6-185.us-west-2.compute.internal:54323 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         07 secs
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    1 month
H2O cluster name:           sparkling-water-livy_application_1579215405741_0005
H2O cluster total nodes:    1
H2O cluster free memory:    8.94 Gb
H2O cluster total cores:    4
H2O cluster allowed cores:  4
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://ip-172-31-6-185.us-west-2.compute.internal:54323
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, Amazon S3, AutoML, Core V3, TargetEncoder, Core V4
Python version:             3.6.8 final
--------------------------  ------------------------------------------------------------

In [ ]:
# import pyspark
# ss = pyspark.sql.SparkSession.builder.master('yarn')\
# .config("spark.dynamicAllocation.enabled", True)\
# .getOrCreate()
# from pysparkling import *
# import h2o
# hc = H2OContext.getOrCreate(ss)

In [ ]:
# fname = "s3://msds697-g2/Wrist/S_2.csv"
# wrist = sc.textFile(fname)
# wrist = ss.read.csv(wrist, header=True, inferSchema=True)

In [4]:
def clean_df(df, sub_id):
    df = df.filter(df.TEMP.isNotNull())
    df = df.withColumn('sub_id', lit(f'S_{sub_id}'))
    df = df.filter(df.label.isin([1,2,3]))
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
chest_path = 's3://msds697-g2/Chest/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
chest = ss.read.csv(chest_path+'S_2.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def clean_df(df):
    df = df.filter(df.TEMP.isNotNull())
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
chest = ss.read.csv(f"{chest_path}S_2.csv", header=True, inferSchema=True)
all_chest = chest.filter(chest.label.isin([1, 2, 3]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
for sub_id in nums[1:]:
    chest = ss.read.csv(f"{chest_path}S_{sub_id}.csv", header=True, inferSchema=True)
    chest = chest.filter(chest.label.isin([1, 2, 3]))
    all_chest = all_chest.union(chest)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
train_data = all_chest.filter(all_chest.sub_id != "S17")
test_data = all_chest.filter(all_chest.sub_id == "S17")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
test_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1593200

## begin h2o

In [12]:
train_h2o = hc.as_h2o_frame(train_data, "train_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
test_h2o = hc.as_h2o_frame(test_data, "test_data")

In [ ]:
train_h2o.show(5)

In [ ]:
train_h2o["label"] = train_h2o["label"].asfactor()

In [ ]:
predictors = train_h2o.names[:]
predictors.remove("col_1")
response = "label"
predictors.remove(response)